# **Digit Recognizer**
This is a idk, like a guide on how to make a nn model that can read doctor's writing

---

### **Preparing the data**

#### Import library

- *Numpy*: for creation of arrays and bring me back the good ol' day of linear algebra
- *Pandas*: for file reading (mainly)
- *Mathplotlib*: for the training process visualization

In [9]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

#### Import Data

We import the data from the csv file using *pd.read_csv()*

In [10]:
data = pd.read_csv('/CODEEEEE/NOTEBOOK/artificial_retard/digit_recognizer/mnist_data.csv')

Use *data.head()* to like see some funny spreadsheet and stuff :D

In [ ]:
data.head()

#### Create Datasets

Turn it into an array, then shuffle the fuck out of it as if you were cooking sum of data gud spicy hot noddle.

In [11]:
data = np.array(data)
m, n = data.shape
#shuffle da data :D
np.random.shuffle(data)

Divide the data into 3 sets:
- *Train set*: to **form** the neural network
- *Validation(dev) set*: to **evaluate** and **tweak** the **hyperparameters**
- *Test set*: to force the model to attend the **university entrance exam** for a chance of getting a **degree** to finally end up in a **job at mc donald**, working **minimum wages**, can't afford **accommodation** for family which inevitably result in **divorce** (you bet your ass that model gonna lose the right for its children in court cuz its vocabulary is limited to only 10 numbers) (its wife is the letter recognizer, the ability to actually construct a meaningful sentence proven to be quite an important asset in convincing someone)

The initial data that we were give has a shape of **({i forgor}, 785)**, or in dummy term: an array with **{i forgor}** values (we shall call each of those value **dat[i]**)

Each of **dat[i]** has 785 values:
- *1st one* being **label** (the answer)
- *The 784 rest* being the **brightness** of each pixel in the handwriting

The idea here is to **seperate** between the **labels** and the **pixel values** and put them into corresponding set:
- *X*: representation of the handwriting [ [hwrite], ..., [hwrite] ]
- *Y*: the label, or correct answer [ ans, ..., ans ]

You can think of the initial data as a **matrix** with **{i forgor} rows, 785 columns**. The initial idea would be to create a **for loop** incrementing through each **dat[i]** and throw them into a placeholder array, but that is going to take like a bocchilion year

Here's what you can do, from a linear algebra standpoint:
- *Transpose the data to get Ys*: doing so will cause the labels to all be put in **one row** which you can extract into your **Ys**
- *Retranspose the data to fix Xs*: Xs on the first transpose has been tainted, so we can just **rotate** it again to untainted it
- *ACTUALLY, don't retranspose the data to fix Xs*: because of the nature of my code, keeping it transposed is going to help with the **dot product** in the upcoming code **(forward prop)**


In [18]:
#train set
data_train = data[0:50000].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255
_, m_train = X_train.shape

#validation set
data_dev = data[50000:55000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255

#test set
data_test = data[55000: m].T
Y_test = data_test[0]
X_test = data_test[1:n]
X_test = X_test / 255

### **The Learning Process**

#### Activation Functions

These functions will be used to determined the brightness of each neurons

In [19]:
def ReLU(x):
    return np.maximum(0, x)

def deriv_ReLU(x): #get derivative
    return x > 0

def softmax(x):
    return np.exp(x) / sum(np.exp(x))

def one_hot(y):
    one_hot_y = np.zeros( (y.size, y.max() + 1) )
    one_hot_y[np.arange(y.size), y] = 1
    one_hot_y = one_hot_y.T
    return one_hot_y

#### Initializing the parameters

Each parameter is an array of many shapes, we use some linear algebra magic (*dot product*) to make every thing fit nicely together

In [20]:
def init_params():
    w1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    w2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return w1, b1, w2, b2

In [23]:
a, b, c, d = init_params()

In [34]:
a.dot(X_train)[0]

array([3.93069715, 2.30476947, 5.66186462, ..., 3.80135777, 0.33769165,
       3.50870718])

In [33]:
b.shape

(10, 1)

#### Train START!!!

The nn train through 3 processes:
- *Forward Pass*: throw the weights and biases thru the nn to determine the prediction
- *Backprobagation*: using the chain rule of calculus (fuck) and gradient descent to determine the changes in the weights and biases
- *Update Parameters*: change the w, b, that's it lol

In [ ]:
def forward_prop(w1, b1, w2, b2, x):
    z1 = w1.dot(x) + b1 # z = weight*x + bias
    a1 = ReLU(z1) # activation = relu(z)
    z2 = w2.dot(a1) + b2
    a2 = softmax(z2) # output
    return z1, a1, z2, a2

def backward_prop(z1, a1, z2, a2, w1, w2, x, y):
    m = y.size
    one_hot_y = one_hot(y)

    dz2 = a2 - one_hot_y
    dw2 = 1 / m * dz2.dot(a1.T)
    db2 = 1 / m * np.sum(dz2)

    dz1 = w2.T.dot(dz2) * deriv_ReLU(z1)
    dw1 = 1 / m * dz2.dot(x.T)
    db1 = 1 / m * np.sum(dz1)

    return dw1, db1, dw2, db2

def update_params(w1, b1, w2, b2, dw1, db1, dw2, db2, alpha):
    w1 -= dw1 * alpha
    b1 -= db1 * alpha
    w2 -= dw2 * alpha
    b2 -= db2 * alpha
    return w1, b1, w2, b2


*[**!**] Try to set the hyperparameters of the grad_descent() function so that the model don't underfit (literally an idiot) or overfit (memorization onetrick)*

In [ ]:
#grad descent
def get_prediction(x):
    return np.argmax(x, 0)

def get_accuracy(predict, y):
    #this get the sum of all correct / sum of all
    return np.sum(predict == y) / y.size

def grad_descent(x, y, alpha=0.1, iteration=500, iter_step=50):
    w1, b1, w2, b2 = init_params()

    for i in range(iteration):
        z1, a1, z2, a2 = forward_prop(w1, b1, w2, b2, x)
        dw1, db1, dw2, db2 = backward_prop(z1, a1, z2, a2, w1, w2, x, y)
        w1, b1, w2, b2 = update_params(w1, b1, w2, b2, dw1, db1, dw2, db2, alpha)

        if i % iter_step == 0:
            print('Iteration: ', i, ' -> ', i + iter_step)
            print('Acc:', get_accuracy( get_prediction(a2), y ) * 100)
    
    return w1, b1, w2, b2

This tiny piece of code here is just to start the training process (through the for loop). After finishing, these 4 coolass mfs will be the final product and going to be use for every future prediction

In [ ]:
w1, b1, w2, b2 = grad_descent(X_train, Y_train)

### **Graduation Day :D**

Here is the code for testing, there are two mode:
- Testing from the *test set*
- Testing from your *own data*

In [ ]:
def make_predict(X, W1, b1, W2, b2):
    _, _, _, a2 = forward_prop(W1, b1, W2, b2, X)
    return get_prediction(a2)

def test_prediction(w1, b1, w2, b2, img_data=None, img_label=None):
    if img_data is None:
        return

    current_image = img_data

    prediction = make_predict(current_image, w1, b1, w2, b2)
    print("Prediction: ", prediction[0])

    if img_label is not None:
        print("Label: ", img_label)

    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

def predict_from_testset(w1, b1, w2, b2, index):
    test_prediction(w1, b1, w2, b2, X_test[:, index, None], Y_test[index])

In [ ]:
predict_from_testset(w1, b1, w2, b2, 800)

In [ ]:
#Your own input
input_file = open('/CODEEEEE/NOTEBOOK/artificial_retard/digit_recognizer/test.txt')

conv = np.loadtxt(input_file, delimiter=',', unpack=False).reshape((-1, 1))

test_prediction(w1, b1, w2, b2, conv)